<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/ClizbeEvaluatingPerformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Weather model

First, load the dataset from the weatherinszeged table from Thinkful's database.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weatherinszeged = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
weatherinszeged

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 17:00:00+00:00,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 18:00:00+00:00,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 19:00:00+00:00,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 20:00:00+00:00,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why?

In [ ]:
weatherinszeged.corr()

,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure
temperature,1.000000,0.992629,-0.632255,0.008957,0.029988,0.392847,NaN,-0.005447
apparenttemperature,0.992629,1.000000,-0.602571,-0.056650,0.029031,0.381718,NaN,-0.000219
humidity,-0.632255,-0.602571,1.000000,-0.224951,0.000735,-0.369173,NaN,0.005454
windspeed,0.008957,-0.056650,-0.224951,1.000000,0.103822,0.100749,NaN,-0.049263
windbearing,0.029988,0.029031,0.000735,0.103822,1.000000,0.047594,NaN,-0.011651
visibility,0.392847,0.381718,-0.369173,0.100749,0.047594,1.000000,NaN,0.059818
loudcover,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pressure,-0.005447,-0.000219,0.005454,-0.049263,-0.011651,0.059818,NaN,1.000000


In [ ]:
weatherdata = weatherinszeged[['temperature', 'humidity', 'windspeed', 'apparenttemperature']]
weatherdata['tempdiff'] = weatherdata['apparenttemperature']-weatherdata['temperature']
weatherdata

,temperature,humidity,windspeed,apparenttemperature,tempdiff
0,9.472222,0.89,14.1197,7.388889,-2.083333
1,9.355556,0.86,14.2646,7.227778,-2.127778
2,9.377778,0.89,3.9284,9.377778,0.000000
3,8.288889,0.83,14.1036,5.944444,-2.344444
4,8.755556,0.83,11.0446,6.977778,-1.777778
...,...,...,...,...,...
96448,26.016667,0.43,10.9963,26.016667,0.000000
96449,24.583333,0.48,10.0947,24.583333,0.000000
96450,22.038889,0.56,8.9838,22.038889,0.000000
96451,21.522222,0.60,10.5294,21.522222,0.000000


In [ ]:
y = weatherdata['tempdiff']
y.head()

0   -2.083333
1   -2.127778
2    0.000000
3   -2.344444
4   -1.777778
Name: tempdiff, dtype: float64

In [ ]:
X = weatherdata.drop(['tempdiff'], axis=1)
X.head()

,temperature,humidity,windspeed,apparenttemperature
0,9.472222,0.89,14.1197,7.388889
1,9.355556,0.86,14.2646,7.227778
2,9.377778,0.89,3.9284,9.377778
3,8.288889,0.83,14.1036,5.944444
4,8.755556,0.83,11.0446,6.977778


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,temperature,humidity,windspeed,apparenttemperature
92119,22.800000,0.62,9.7405,22.800000
54898,-8.700000,0.74,14.5383,-15.005556
60235,12.966667,0.89,7.4060,12.966667
8803,15.066667,0.50,10.8031,15.066667
44215,10.955556,0.62,15.8746,10.955556


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

temperature           -1.000000e+00
humidity               1.984293e-15
windspeed              1.101907e-16
apparenttemperature    1.000000e+00
dtype: float64

In [ ]:
results.tvalues

temperature           -5.137333e+16
humidity               4.062530e+01
windspeed              3.252096e+01
apparenttemperature    5.633238e+16
dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               tempdiff   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          2.461e+33
Date:                Wed, 24 Jun 2020   Prob (F-statistic):                        0.00
Time:                        01:38:58   Log-Likelihood:                      2.4229e+06
No. Observations:               77162   AIC:                                 -4.846e+06
Df Residuals:                   77158   BIC:                                 -4.846e+06
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
temperature            -1.0000   1.95e-17  -5.14e+16      0.000      -1.000      -1.000
humidity             1.984e-15   4.88e-17     40.625      0.000    1.89e-15    2.08e-15
windspeed            1.102e-16   3.39e-18     32.521      0.000    1.04e-16    1.17e-16
apparenttemperature     1.0000   1.78e-17   5.63e+16      0.000       1.000       1.000
==============================================================================
Omnibus:                     4580.982   Durbin-Watson:                   1.307
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5453.089
Skew:                           0.651   Prob(JB):                         0.00
Kurtosis:                       3.049   Cond. No.                         58.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one?

In [ ]:
weatherdata['weather'] = weatherdata['humidity']*weatherdata['windspeed']
weatherdata

,temperature,humidity,windspeed,apparenttemperature,tempdiff,weather
0,9.472222,0.89,14.1197,7.388889,-2.083333,12.566533
1,9.355556,0.86,14.2646,7.227778,-2.127778,12.267556
2,9.377778,0.89,3.9284,9.377778,0.000000,3.496276
3,8.288889,0.83,14.1036,5.944444,-2.344444,11.705988
4,8.755556,0.83,11.0446,6.977778,-1.777778,9.167018
...,...,...,...,...,...,...
96448,26.016667,0.43,10.9963,26.016667,0.000000,4.728409
96449,24.583333,0.48,10.0947,24.583333,0.000000,4.845456
96450,22.038889,0.56,8.9838,22.038889,0.000000,5.030928
96451,21.522222,0.60,10.5294,21.522222,0.000000,6.317640


In [ ]:
y = weatherdata['weather']
y.head()

0    12.566533
1    12.267556
2     3.496276
3    11.705988
4     9.167018
Name: weather, dtype: float64

In [ ]:
X = weatherdata.drop(['weather'], axis=1)
X.head()

,temperature,humidity,windspeed,apparenttemperature,tempdiff
0,9.472222,0.89,14.1197,7.388889,-2.083333
1,9.355556,0.86,14.2646,7.227778,-2.127778
2,9.377778,0.89,3.9284,9.377778,0.000000
3,8.288889,0.83,14.1036,5.944444,-2.344444
4,8.755556,0.83,11.0446,6.977778,-1.777778


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,temperature,humidity,windspeed,apparenttemperature,tempdiff
47396,1.088889,0.86,22.9908,-4.244444,-5.333333
57534,16.061111,0.87,7.8246,16.061111,0.000000
17247,23.883333,0.32,7.7441,23.883333,0.000000
86275,8.344444,0.96,26.5006,4.705556,-3.638889
25222,12.244444,0.73,10.9319,12.244444,0.000000


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

temperature           -0.030500
humidity               3.194052
windspeed              0.608160
apparenttemperature   -0.073465
tempdiff              -0.042964
dtype: float64

In [ ]:
results.tvalues

temperature            -13.768905
humidity               210.736785
windspeed              575.249734
apparenttemperature    -43.524266
tempdiff               -11.118561
dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                weather   R-squared (uncentered):                   0.964
Model:                            OLS   Adj. R-squared (uncentered):              0.964
Method:                 Least Squares   F-statistic:                          5.128e+05
Date:                Wed, 24 Jun 2020   Prob (F-statistic):                        0.00
Time:                        01:38:58   Log-Likelihood:                     -1.5215e+05
No. Observations:               77162   AIC:                                  3.043e+05
Df Residuals:                   77158   BIC:                                  3.043e+05
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
temperature            -0.0305      0.002    -13.769      0.000      -0.035      -0.026
humidity                3.1941      0.015    210.737      0.000       3.164       3.224
windspeed               0.6082      0.001    575.250      0.000       0.606       0.610
apparenttemperature    -0.0735      0.002    -43.524      0.000      -0.077      -0.070
tempdiff               -0.0430      0.004    -11.119      0.000      -0.051      -0.035
==============================================================================
Omnibus:                     8828.616   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            75491.032
Skew:                          -0.220   Prob(JB):                         0.00
Kurtosis:                       7.826   Cond. No.                     2.67e+16
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.89e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Add visibility as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the visibility in terms of the improvement in the adjusted R-squared. Which one is more useful?

In [ ]:
weatherdata = weatherinszeged[['temperature', 'humidity', 'windspeed', 'apparenttemperature', 'visibility']]
weatherdata['tempdiff'] = weatherdata['apparenttemperature']-weatherdata['temperature']
weatherdata

,temperature,humidity,windspeed,apparenttemperature,visibility,tempdiff
0,9.472222,0.89,14.1197,7.388889,15.8263,-2.083333
1,9.355556,0.86,14.2646,7.227778,15.8263,-2.127778
2,9.377778,0.89,3.9284,9.377778,14.9569,0.000000
3,8.288889,0.83,14.1036,5.944444,15.8263,-2.344444
4,8.755556,0.83,11.0446,6.977778,15.8263,-1.777778
...,...,...,...,...,...,...
96448,26.016667,0.43,10.9963,26.016667,16.1000,0.000000
96449,24.583333,0.48,10.0947,24.583333,15.5526,0.000000
96450,22.038889,0.56,8.9838,22.038889,16.1000,0.000000
96451,21.522222,0.60,10.5294,21.522222,16.1000,0.000000


In [ ]:
y = weatherdata['tempdiff']
y.head()

0   -2.083333
1   -2.127778
2    0.000000
3   -2.344444
4   -1.777778
Name: tempdiff, dtype: float64

In [ ]:
X = weatherdata.drop(['tempdiff'], axis=1)
X.head()

,temperature,humidity,windspeed,apparenttemperature,visibility
0,9.472222,0.89,14.1197,7.388889,15.8263
1,9.355556,0.86,14.2646,7.227778,15.8263
2,9.377778,0.89,3.9284,9.377778,14.9569
3,8.288889,0.83,14.1036,5.944444,15.8263
4,8.755556,0.83,11.0446,6.977778,15.8263


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,temperature,humidity,windspeed,apparenttemperature,visibility
29783,0.555556,0.82,14.4900,-3.616667,16.1000
4878,21.066667,0.93,5.1198,21.066667,9.9015
62401,21.483333,0.48,6.4400,21.483333,16.1000
44436,18.933333,0.31,4.7817,18.933333,9.9820
91053,-0.600000,1.00,6.1663,-2.766667,0.1610


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

temperature           -1.000000e+00
humidity               1.043219e-15
windspeed             -2.253768e-16
apparenttemperature    1.000000e+00
visibility             1.153388e-16
dtype: float64

In [ ]:
results.tvalues

temperature           -2.036386e+16
humidity               7.333985e+00
windspeed             -2.648992e+01
apparenttemperature    2.257360e+16
visibility             9.537322e+00
dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:               tempdiff   R-squared (uncentered):                   1.000
Model:                            OLS   Adj. R-squared (uncentered):              1.000
Method:                 Least Squares   F-statistic:                          3.241e+32
Date:                Wed, 24 Jun 2020   Prob (F-statistic):                        0.00
Time:                        01:38:58   Log-Likelihood:                      2.3532e+06
No. Observations:               77162   AIC:                                 -4.706e+06
Df Residuals:                   77157   BIC:                                 -4.706e+06
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
temperature            -1.0000   4.91e-17  -2.04e+16      0.000      -1.000      -1.000
humidity             1.043e-15   1.42e-16      7.334      0.000    7.64e-16    1.32e-15
windspeed           -2.254e-16   8.51e-18    -26.490      0.000   -2.42e-16   -2.09e-16
apparenttemperature     1.0000   4.43e-17   2.26e+16      0.000       1.000       1.000
visibility           1.153e-16   1.21e-17      9.537      0.000    9.16e-17    1.39e-16
==============================================================================
Omnibus:                     1775.031   Durbin-Watson:                   0.897
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1018.759
Skew:                           0.113   Prob(JB):                    6.02e-222
Kurtosis:                       2.484   Cond. No.                         73.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.

2. House prices model

Load the houseprices data from Thinkful's database.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
houseprices

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,lotconfig,landslope,neighborhood,condition1,condition2,bldgtype,housestyle,overallqual,overallcond,yearbuilt,yearremodadd,roofstyle,roofmatl,exterior1st,exterior2nd,masvnrtype,masvnrarea,exterqual,extercond,foundation,bsmtqual,bsmtcond,bsmtexposure,bsmtfintype1,bsmtfinsf1,bsmtfintype2,bsmtfinsf2,bsmtunfsf,totalbsmtsf,heating,...,centralair,electrical,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,kitchenqual,totrmsabvgrd,functional,fireplaces,fireplacequ,garagetype,garageyrblt,garagefinish,garagecars,garagearea,garagequal,garagecond,paveddrive,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,None,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,None,None,None,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,None,None,None,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,None,MnPrv,None,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,None,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,None,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,

In [ ]:
housedata = houseprices[['salecondition', 'saleprice', 'yearbuilt' , 'housestyle', 'lotarea', 'lotfrontage']]

In [ ]:
housedata.head()

,salecondition,saleprice,yearbuilt,housestyle,lotarea,lotfrontage
0,Normal,208500,2003,2Story,8450,65.0
1,Normal,181500,1976,1Story,9600,80.0
2,Normal,223500,2001,2Story,11250,68.0
3,Abnorml,140000,1915,2Story,9550,60.0
4,Normal,250000,2000,2Story,14260,84.0


In [ ]:
housedata = pd.get_dummies(houseprices, dummy_na=True)
housedata.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,mszoning_C (all),mszoning_FV,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
housedata.dropna(inplace=True)

In [ ]:
y = housedata['saleprice']

X = housedata.drop('saleprice', axis=1)
X.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: saleprice, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
516,516,20,94.0,12220,10,5,2009,2009,305.0,1436,0,570,2006,2020,0,0,2020,1,0,2,1,3,1,9,1,2009.0,3,900,156,54,0,0,0,0,0,9,2009,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
739,740,60,65.0,9313,7,5,2004,2004,0.0,0,0,864,864,864,864,0,1728,0,0,2,1,3,1,7,0,2004.0,2,572,187,56,0,0,0,0,0,4,2009,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1185,1186,50,60.0,9738,5,7,1924,1950,0.0,392,0,392,784,949,272,0,1221,1,0,1,0,4,1,7,0,1965.0,1,392,0,0,236,0,0,0,0,3,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
71,72,20,69.0,7599,4,6,1982,2006,0.0,565,0,280,845,845,0,0,845,1,0,1,0,2,1,4,0,1987.0,2,360,0,0,0,0,0,0,0,6,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
131,131,60,88.0,14200,7,6,1966,1966,309.0,445,0,479,924,1216,941,0,2157,0,0,2,1,4,1,8,2,1966.0,2,487,105,66,0,0,0,0,0,5,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

id                           2.318431
mssubclass                 -10.581120
lotfrontage                116.949375
lotarea                      0.533548
overallqual               7336.389556
                             ...     
salecondition_Alloca    -23079.814329
salecondition_Family    -34227.094742
salecondition_Normal    -23998.875473
salecondition_Partial   -19546.617182
salecondition_nan            0.000000
Length: 332, dtype: float64

In [ ]:
results.tvalues

id                       1.056425
mssubclass              -0.071333
lotfrontage              1.643350
lotarea                  1.682517
overallqual              4.902216
                           ...   
salecondition_Alloca    -0.811533
salecondition_Family    -1.280633
salecondition_Normal    -0.907248
salecondition_Partial   -0.654833
salecondition_nan             NaN
Length: 332, dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.922
Method:                 Least Squares   F-statistic:                     45.79
Date:                Wed, 24 Jun 2020   Prob (F-statistic):          2.22e-302
Time:                        01:39:00   Log-Likelihood:                -10173.
No. Observations:                 896   AIC:                         2.082e+04
Df Residuals:                     660   BIC:                         2.195e+04
Df Model:                         235                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
id                        2.3184      2.195      1.056      0.291      -1.991       6.628
mssubclass              -10.5811    148.334     -0.071      0.943    -301.844     280.682
lotfrontage             116.9494     71.165      1.643      0.101     -22.788     256.687
lotarea                   0.5335      0.317      1.683      0.093      -0.089       1.156
overallqual            7336.3896   1496.545      4.902      0.000    4397.826    1.03e+04
overallcond            6139.7404   1359.480      4.516      0.000    3470.313    8809.168
yearbuilt               116.8762    120.594      0.969      0.333    -119.917     353.670
yearremodadd            112.7612     88.561      1.273      0.203     -61.134     286.657
masvnrarea               24.9237      7.749      3.216      0.001       9.707      40.140
bsmtfinsf1               19.8786      4.429      4.489      0.000      11.183      28.574
bsmtfinsf2               11.0179      9.294      1.185      0.236      -7.231      29.267
bsmtunfsf                -2.0708      4.319     -0.480      0.632     -10.551       6.409
totalbsmtsf              28.8256      6.487      4.443      0.000      16.087      41.564
firstflrsf                6.8273     11.861      0.576      0.565     -16.463      30.118
secondflrsf              29.0977      9.541      3.050      0.002      10.362      47.833
lowqualfinsf             -6.5590     27.636     -0.237      0.812     -60.824      47.706
grlivarea                29.3660     10.374      2.831      0.005       8.997      49.735
bsmtfullbath          -1188.2574   2975.744     -0.399      0.690   -7031.323    4654.808
bsmthalfbath          -2112.0123   4439.569     -0.476      0.634   -1.08e+04    6605.370
fullbath               6606.8105   3446.353      1.917      0.056    -160.327    1.34e+04
halfbath               5025.0882   3091.596      1.625      0.105   -1045.462    1.11e+04
bedroomabvgr          -2906.9406   2092.929     -1.389      0.165   -7016.543    1202.662
kitchenabvgr          -1.344e+04   9818.264     -1.369      0.172   -3.27e+04    5841.386
totrmsabvgrd           1593.1422   1403.400      1.135      0.257   -1162.525    4348.809
fireplaces             5280.5017   3815.000      1.384      0.167   -2210.498    1.28e+04
garageyrblt             -15.0116     90.122     -0.167      0.868    -191.972     161.948
garagecars             6682.4574   3265.540      2.046      0.041     270.358    1.31e+04
garagearea               11.0157     11.150      0.988      0.324     -10.878      32.909
wooddecksf               15.8216      8.507      1.860      0.063      -0.882      32.525
openporchsf             -11.1791     19.408     -0.576      0.565     -49.289      26.931
enclosedporch             6.0421     19.654      0.307      0.759     -32.550      44.634
threessnporch          

Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

In [ ]:
housedata = houseprices[['salecondition', 'saleprice', 'yearbuilt' , 'housestyle', 'lotarea', 'lotfrontage']]

In [ ]:
housedata = pd.get_dummies(houseprices, dummy_na=True)
housedata.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,mszoning_C (all),mszoning_FV,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
housedata.dropna(inplace=True)

In [ ]:
y = housedata['saleprice']

X = housedata.drop('saleprice', axis=1)
X.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: saleprice, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,id,mssubclass,lotfrontage,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
688,689,20,60.0,8089,8,6,2007,2007,0.0,945,0,474,1419,1419,0,0,1419,1,0,2,0,2,1,7,1,2007.0,2,567,140,0,0,0,0,0,0,10,2007,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1067,1068,60,80.0,9760,6,6,1964,1964,360.0,674,106,0,780,798,813,0,1611,1,0,1,1,4,1,7,0,1964.0,2,442,328,128,0,0,189,0,0,6,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
674,675,20,80.0,9200,6,6,1965,1965,0.0,892,0,244,1136,1136,0,0,1136,1,0,1,0,3,1,5,1,1965.0,1,384,426,0,0,0,0,0,0,7,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
656,657,20,72.0,10007,5,7,1959,2006,54.0,806,0,247,1053,1053,0,0,1053,1,0,1,1,3,1,5,0,1959.0,1,312,0,0,0,0,0,0,0,8,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
744,745,120,41.0,5395,8,5,1993,1993,0.0,733,0,604,1337,1337,0,0,1337,1,0,2,0,2,1,5,1,1993.0,2,462,96,0,70,168,0,0,0,10,2008,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

id                           2.749027
mssubclass                -103.643123
lotfrontage                 54.869291
lotarea                      0.654615
overallqual               7315.475247
                             ...     
salecondition_Alloca    -38100.294958
salecondition_Family    -39384.999423
salecondition_Normal    -38350.020563
salecondition_Partial   -12252.989138
salecondition_nan            0.000000
Length: 332, dtype: float64

In [ ]:
results.tvalues

id                       1.240078
mssubclass              -0.753204
lotfrontage              0.757822
lotarea                  3.705076
overallqual              4.710962
                           ...   
salecondition_Alloca    -1.372129
salecondition_Family    -1.498132
salecondition_Normal    -1.459494
salecondition_Partial   -0.370016
salecondition_nan             NaN
Length: 332, dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     45.44
Date:                Wed, 24 Jun 2020   Prob (F-statistic):          1.16e-301
Time:                        01:40:10   Log-Likelihood:                -10175.
No. Observations:                 896   AIC:                         2.081e+04
Df Residuals:                     665   BIC:                         2.192e+04
Df Model:                         230                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
id                        2.7490      2.217      1.240      0.215      -1.604       7.102
mssubclass             -103.6431    137.603     -0.753      0.452    -373.832     166.545
lotfrontage              54.8693     72.404      0.758      0.449     -87.299     197.037
lotarea                   0.6546      0.177      3.705      0.000       0.308       1.002
overallqual            7315.4752   1552.862      4.711      0.000    4266.372    1.04e+04
overallcond            5362.3832   1365.748      3.926      0.000    2680.685    8044.081
yearbuilt               207.6768    119.608      1.736      0.083     -27.178     442.531
yearremodadd            123.2210     83.125      1.482      0.139     -39.999     286.441
masvnrarea               23.0945      7.602      3.038      0.002       8.169      38.020
bsmtfinsf1               15.6026      4.079      3.825      0.000       7.593      23.612
bsmtfinsf2                7.7026      8.790      0.876      0.381      -9.557      24.962
bsmtunfsf                 1.4940      4.073      0.367      0.714      -6.503       9.491
totalbsmtsf              24.7993      6.390      3.881      0.000      12.253      37.346
firstflrsf                4.1036     12.874      0.319      0.750     -21.176      29.383
secondflrsf              20.7208     10.841      1.911      0.056      -0.565      42.007
lowqualfinsf             11.7974     31.542      0.374      0.709     -50.137      73.732
grlivarea                36.6217     11.683      3.135      0.002      13.682      59.562
bsmtfullbath           2637.7091   2871.652      0.919      0.359   -3000.888    8276.306
bsmthalfbath           2248.1399   4446.661      0.506      0.613   -6483.047     1.1e+04
fullbath               7317.0315   3298.918      2.218      0.027     839.481    1.38e+04
halfbath               4375.8090   3142.863      1.392      0.164   -1795.321    1.05e+04
bedroomabvgr          -3436.7294   2056.875     -1.671      0.095   -7475.481     602.022
kitchenabvgr          -1.645e+04   1.04e+04     -1.581      0.114   -3.69e+04    3976.282
totrmsabvgrd           1052.3945   1368.772      0.769      0.442   -1635.241    3740.030
fireplaces             6844.9257   3984.478      1.718      0.086    -978.747    1.47e+04
garageyrblt             -58.8460     89.630     -0.657      0.512    -234.837     117.145
garagecars             1998.7178   3179.712      0.629      0.530   -4244.768    8242.203
garagearea               27.0570     11.594      2.334      0.020       4.292      49.822
wooddecksf               15.6686      8.568      1.829      0.068      -1.155      32.492
openporchsf               0.6952     17.823      0.039      0.969     -34.302      35.692
enclosedporch             5.5873     19.390      0.288      0.773     -32.487      43.661
threessnporch          

Do you think your model is satisfactory? If so, why?

In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.

In [ ]:
housedata = housedata.drop('lotfrontage', axis=1)
housedata

,id,mssubclass,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,mszoning_C (all),mszoning_FV,mszoning_RH,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,208500,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,181500,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,223500,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,140000,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,250000,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0.0,0,0,953,953,953,694,0,1647,0,0,2,1,3,1,7,1,1999.0,2,460,0,40,0,0,0,0,0,8,2007,175000,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1456,1457,20,13175,6,6,1978,1988,119.0,790,163,589,1542,2073,0,0,2073,1,0,2,0,3,1,7,2,1978.0,2,500,349,0,0,0,0,0,0,2,2010,210000,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1457,1458,70,9042,7,9,1941,2006,0.0,275,0,877,1152,1188,1152,0,2340,0,0,2,0,4,1,9,2,1941.0,1,252,0,60,0,0,0,0,2500,5,2010,266500,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1458,1459,20,9717,5,6,1950,1996,0.0,49,1029,0,1078,1078,0,0,1078,1,0,1,0,2,1,5,0,1950.0,1,240,366,0,112,0,0,0,0,4,2010,142125,0,0,0,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
y = housedata['saleprice']

X = housedata.drop('saleprice', axis=1)
X.head()

,id,mssubclass,lotarea,overallqual,overallcond,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,secondflrsf,lowqualfinsf,grlivarea,bsmtfullbath,bsmthalfbath,fullbath,halfbath,bedroomabvgr,kitchenabvgr,totrmsabvgrd,fireplaces,garageyrblt,garagecars,garagearea,wooddecksf,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,mszoning_C (all),mszoning_FV,mszoning_RH,mszoning_RL,...,garagecond_Fa,garagecond_Gd,garagecond_Po,garagecond_TA,garagecond_nan,paveddrive_N,paveddrive_P,paveddrive_Y,paveddrive_nan,poolqc_Ex,poolqc_Fa,poolqc_Gd,poolqc_nan,fence_GdPrv,fence_GdWo,fence_MnPrv,fence_MnWw,fence_nan,miscfeature_Gar2,miscfeature_Othr,miscfeature_Shed,miscfeature_TenC,miscfeature_nan,saletype_COD,saletype_CWD,saletype_Con,saletype_ConLD,saletype_ConLI,saletype_ConLw,saletype_New,saletype_Oth,saletype_WD,saletype_nan,salecondition_Abnorml,salecondition_AdjLand,salecondition_Alloca,salecondition_Family,salecondition_Normal,salecondition_Partial,salecondition_nan
0,1,60,8450,7,5,2003,2003,196.0,706,0,150,856,856,854,0,1710,1,0,2,1,3,1,8,0,2003.0,2,548,0,61,0,0,0,0,0,2,2008,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
1,2,20,9600,6,8,1976,1976,0.0,978,0,284,1262,1262,0,0,1262,0,1,2,0,3,1,6,1,1976.0,2,460,298,0,0,0,0,0,0,5,2007,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,3,60,11250,7,5,2001,2002,162.0,486,0,434,920,920,866,0,1786,1,0,2,1,3,1,6,1,2001.0,2,608,0,42,0,0,0,0,0,9,2008,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
3,4,70,9550,7,5,1915,1970,0.0,216,0,540,756,961,756,0,1717,1,0,1,0,3,1,7,1,1998.0,3,642,0,35,272,0,0,0,0,2,2006,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0
4,5,60,14260,8,5,2000,2000,350.0,655,0,490,1145,1145,1053,0,2198,1,0,2,1,4,1,9,1,2000.0,3,836,192,84,0,0,0,0,0,12,2008,0,0,0,1,...,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
y.head()

0    208500
1    181500
2    223500
3    140000
4    250000
Name: saleprice, dtype: int64

In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

id                           2.749027
mssubclass                -103.643123
lotfrontage                 54.869291
lotarea                      0.654615
overallqual               7315.475247
                             ...     
salecondition_Alloca    -38100.294958
salecondition_Family    -39384.999423
salecondition_Normal    -38350.020563
salecondition_Partial   -12252.989138
salecondition_nan            0.000000
Length: 332, dtype: float64

In [ ]:
results.tvalues

id                       1.240078
mssubclass              -0.753204
lotfrontage              0.757822
lotarea                  3.705076
overallqual              4.710962
                           ...   
salecondition_Alloca    -1.372129
salecondition_Family    -1.498132
salecondition_Normal    -1.459494
salecondition_Partial   -0.370016
salecondition_nan             NaN
Length: 332, dtype: float64

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.940
Model:                            OLS   Adj. R-squared:                  0.919
Method:                 Least Squares   F-statistic:                     45.44
Date:                Wed, 24 Jun 2020   Prob (F-statistic):          1.16e-301
Time:                        01:40:36   Log-Likelihood:                -10175.
No. Observations:                 896   AIC:                         2.081e+04
Df Residuals:                     665   BIC:                         2.192e+04
Df Model:                         230                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
id                        2.7490      2.217      1.240      0.215      -1.604       7.102
mssubclass             -103.6431    137.603     -0.753      0.452    -373.832     166.545
lotfrontage              54.8693     72.404      0.758      0.449     -87.299     197.037
lotarea                   0.6546      0.177      3.705      0.000       0.308       1.002
overallqual            7315.4752   1552.862      4.711      0.000    4266.372    1.04e+04
overallcond            5362.3832   1365.748      3.926      0.000    2680.685    8044.081
yearbuilt               207.6768    119.608      1.736      0.083     -27.178     442.531
yearremodadd            123.2210     83.125      1.482      0.139     -39.999     286.441
masvnrarea               23.0945      7.602      3.038      0.002       8.169      38.020
bsmtfinsf1               15.6026      4.079      3.825      0.000       7.593      23.612
bsmtfinsf2                7.7026      8.790      0.876      0.381      -9.557      24.962
bsmtunfsf                 1.4940      4.073      0.367      0.714      -6.503       9.491
totalbsmtsf              24.7993      6.390      3.881      0.000      12.253      37.346
firstflrsf                4.1036     12.874      0.319      0.750     -21.176      29.383
secondflrsf              20.7208     10.841      1.911      0.056      -0.565      42.007
lowqualfinsf             11.7974     31.542      0.374      0.709     -50.137      73.732
grlivarea                36.6217     11.683      3.135      0.002      13.682      59.562
bsmtfullbath           2637.7091   2871.652      0.919      0.359   -3000.888    8276.306
bsmthalfbath           2248.1399   4446.661      0.506      0.613   -6483.047     1.1e+04
fullbath               7317.0315   3298.918      2.218      0.027     839.481    1.38e+04
halfbath               4375.8090   3142.863      1.392      0.164   -1795.321    1.05e+04
bedroomabvgr          -3436.7294   2056.875     -1.671      0.095   -7475.481     602.022
kitchenabvgr          -1.645e+04   1.04e+04     -1.581      0.114   -3.69e+04    3976.282
totrmsabvgrd           1052.3945   1368.772      0.769      0.442   -1635.241    3740.030
fireplaces             6844.9257   3984.478      1.718      0.086    -978.747    1.47e+04
garageyrblt             -58.8460     89.630     -0.657      0.512    -234.837     117.145
garagecars             1998.7178   3179.712      0.629      0.530   -4244.768    8242.203
garagearea               27.0570     11.594      2.334      0.020       4.292      49.822
wooddecksf               15.6686      8.568      1.829      0.068      -1.155      32.492
openporchsf               0.6952     17.823      0.039      0.969     -34.302      35.692
enclosedporch             5.5873     19.390      0.288      0.773     -32.487      43.661
threessnporch          

For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?